# Advanced Example: Using a Different Alternative Investment
One obvious question about the previous example and project was the choice of a risk-free alternative investment. Perhaps users would be more interested in training agents to consider other alternatives. Furthermore, the risk adjustment in the project measured risk relative to market performance, represented by `SPY` returns and volatility using $\beta$, but compared the risk-adjusted performance with a risk-free alternative. Perhaps a more natural choice for both issues would be to use `SPY` as the alternative investment.

In this example, we change the alternative investment to `SPY` and compare two cases: a risk-blind case where we do not use the $\beta^{-\gamma}$ formulation and compare tickers versus `SPY,` and a risk-adjusted case where we do correct for risk and compare risk-adjusted ticker performance versus `SPY.`

## Learning objectives
This example's objective is to explore the use of `SPY` as an alternative investment in risk-blind and risk-adjusted scenarios. 
* __Prerequisites__: Load the ticker-picker save file from the worked example and set constant values that we use in the later tasks.
* __Task 1__: Compute the risk measure dictionary for my ticker list.
    * `TODO`: Compute the covariance for firms in the my_tickers list
    * `TODO`: Compute the risk-measure dictionary
* __Task 2__:  Build a risk-aware ticker-picker agent and world models
    * `TODO`: Setup a MyEpsilonSamplingBanditModel instance
    * `TODO`: Setup a MyTickerPickerRiskAwareWorldModel instance
* __Task 3__: Compute the preferences of a population risk-aware ticker-picker agents
    * `TODO`: Compute the wisdom of the risk-aware collective
    * `TODO`: Preference table for risk-blind and risk-aware agents

## Setup
We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our lab problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl).

In [3]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CH

## Prerequisites: Load ticker-picker save file
Let's begin by loading the [HDF5 encoded saved file](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) generated in the worked example using [a `load(...)`  method exported by the JLD2.jl package](https://github.com/JuliaIO/JLD2.jl). First, we specify the path to the saved file in the `path_to_save_file::String` variable:

In [5]:
number_of_agents = 10000; # how many agents do we want
path_to_save_file = joinpath(_PATH_TO_DATA, "TickerPickerAgent-Simulation-State-ALT-RB-SPY-N-$(number_of_agents).jld2");

then [call the `load(...)` method exported by the JLD2.jl package](https://github.com/JuliaIO/JLD2.jl), which reads the binary saved file and returns the saved data as a dictionary; we assign the data to the `saved_state_dict::Dict{String, Any}` variable:

In [7]:
saved_state_dict = load(path_to_save_file)

Dict{String, Any} with 10 entries:
  "risk_free_rate"              => 0.047
  "maximum_number_trading_days" => 1508
  "M"                           => 10
  "buffersize"                  => 5
  "data"                        => Dict{String, DataFrame}("NI"=>1508×8 DataFra…
  "preferences"                 => Beta[Beta{Float64}(α=1.0, β=3.0) Beta{Float6…
  "tickers"                     => ["AAPL", "AMD", "BAC", "C", "CMCSA", "ECL", …
  "probability_top_dictionary"  => Dict("JPM"=>0.3659, "MSFT"=>0.5852, "C"=>0.2…
  "banditmodel"                 => MyEpsilonSamplingBanditModel([1.0, 1.0, 1.0,…
  "Δt"                          => 0.00396825

### Set constant values
In the code block below, we set various constant values (and compute some derived constants) that we use in the tasks below. We generally access these values from the `saved_state_dict::Dict{String, Any}` by passing a `key::String` value to the saved data dictionary. Please see the comment beside the code line for more details about the value and what we'll use it for.

In [9]:
dataset = saved_state_dict["data"]; # OHLC data 460 or so firms/ETFs
my_tickers = saved_state_dict["tickers"] |> sort; # *sorted* list of tickers we specified in the worked example
Δt = saved_state_dict["Δt"]; # time step in the data
maximum_number_trading_days = saved_state_dict["maximum_number_trading_days"]; # how many trading days is in the clean data
risk_free_rate = saved_state_dict["risk_free_rate"]; # what is the risk-free rate we used in the example
risk_blind_probability_top_dictionary = saved_state_dict["probability_top_dictionary"]; # probability results from worked example
risk_blind_preferences = saved_state_dict["preferences"]; # prefs in the risk-blind case

index_market = findfirst(x -> x == "SPY", my_tickers); # what is the index of SPY in my_tickers (we use this for Task 1)
K = length(my_tickers); # how tickers (arms) do we have? 

## Task 1: Compute the risk measure dictionary for my ticker list
In this task, let's compute a dictionary of [beta values](https://en.wikipedia.org/wiki/Beta_(finance)) that we'll use later to make our agents risk-aware. First, we'll compute the growth rates and the [covariance matrix](https://en.wikipedia.org/wiki/Covariance_matrix) and then use these values to develop the risk measure dictionary.

### TODO: Compute the covariance for firms in the my_tickers list
The covariance matrix $\Sigma$ will be used to estimate $\beta$ values for the `tickers`$\in$`my_tickers.` 

* First, compute the expected (annualized) log growth rate by passing the `dataset::Dict{String, DataFrame}` and the `my_tickers::Array{String,1}` list to the [log_growth_matrix(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.log_growth_matrix).
* Store the growth rates in the `all_firms_return_matrix` variable, a $T-1\times{K}$ array of log growth values. Each row of `all_firms_return_matrix` corresponds to a time value, while each column corresponds to a firm

In [12]:
all_firms_return_matrix = log_growth_matrix(dataset, my_tickers, Δt = Δt, 
    risk_free_rate = 0.0); # *not* excess growth, so we set risk_free_rate = 0

Next, estimate the annualized `covariance_matrix` (assuming `252` trading days per year) from the `all_firms_return_matrix` using the [cov(...) function](https://docs.julialang.org/en/v1/stdlib/Statistics/#Statistics.cov), exported by the [Statistics.jl package](https://docs.julialang.org/en/v1/stdlib/Statistics/). We store the $K\times{K}$ covariance matrix in the $\Sigma$ variable:

In [14]:
Σ = cov(all_firms_return_matrix) |> x -> x*(1/252) # annualized, historical volatility

29×29 Matrix{Float64}:
 0.0691322   0.0594079   0.0302666   …  0.0686136   0.02538    0.0262735
 0.0594079   0.211496    0.0425124      0.0995237   0.0284595  0.036321
 0.0302666   0.0425124   0.0894146      0.0498514   0.0298376  0.0780059
 0.0351969   0.050067    0.0849407      0.0590759   0.0327878  0.080665
 0.0224277   0.0281091   0.0314937      0.0299769   0.0183759  0.0308962
 0.0306041   0.040796    0.0414892   …  0.0451837   0.0265421  0.0396317
 0.0319759   0.0452737   0.0676833      0.0500259   0.0276715  0.0632671
 0.0380052   0.0536885   0.035675       0.0549636   0.0192692  0.0330759
 0.0153932   0.0129632   0.0166998      0.0130472   0.0189617  0.0158655
 0.0269166   0.0354797   0.0715436      0.0429108   0.0277043  0.0667961
 0.00473437  0.00336195  0.00283112  …  0.00182335  0.0033409  0.00247071
 0.0167374   0.0164346   0.0247161      0.0156101   0.0204447  0.0254018
 0.0297858   0.0378785   0.0742538      0.0438053   0.0323739  0.0723559
 ⋮                           

### TODO: Compute the risk-measure dictionary
Now, compute the $\beta$ values, using [a `for-loop`](https://docs.julialang.org/en/v1/base/base/#for), for each `ticker`$\in$ `my_tickers` and store these values in the `β::Dict{String, Float64}` dictionary, where the keys are the ticker symbol and the values are the computed $\beta$ values.
* Iterate over the `K`-tickers (arms) in our collection, access a `ticker` from the `my_tickers::Array{String,1}` array, and then get the covariance of the `ticker` with respect to the market (in this case `SPY`), save this in the `σᵢ::Float64` variable,  and save the variance of the market in the `σₘ::Float64` variable. Note: we previously stored the market index in the `index_market::Int64` variable.
* Save the [estimated beta value](https://en.wikipedia.org/wiki/Beta_(finance)) (the ratio `σᵢ/σₘ`) in the `β::Dict{String, Float64}` dictionary, where the key is `ticker` and the value is the ratio `σᵢ/σₘ`.

In [16]:
β = Dict{String, Float64}();
for i ∈ 1:K
    ticker = my_tickers[i];
    σᵢ = Σ[i,index_market];
    σₘ = Σ[index_market,index_market]
    β[ticker] = σᵢ/σₘ;
end
β

Dict{String, Float64} with 29 entries:
  "JPM"   => 1.17336
  "MSFT"  => 1.13379
  "C"     => 1.48297
  "MRK"   => 0.504918
  "UNH"   => 0.878591
  "SPY"   => 1.0
  "BAC"   => 1.32778
  "MU"    => 1.52694
  "SRE"   => 0.794775
  "TSLA"  => 1.77511
  "KR"    => 0.194428
  "AMD"   => 1.67809
  "INTC"  => 1.12753
  "NVDA"  => 1.78828
  "MMM"   => 0.938743
  "NOC"   => 0.566008
  "ECL"   => 1.13742
  "LMT"   => 0.67081
  "GS"    => 1.26915
  "OXY"   => 1.54752
  "CMCSA" => 0.856908
  "JNJ"   => 0.548523
  "SPYD"  => 1.03756
  "NFLX"  => 1.07208
  "WFC"   => 1.25441
  ⋮       => ⋮

## Task 2:  Build a risk-aware ticker-picker agent and world models
In this task, you will build a bandit agent model and a risk-aware world model instance. We will use these models in `task 3` to train a population of risk-aware agents. Let's begin by building a bandit agent model, which is identical to the worked example. Then, we consider the world model, which is modified to account for the presence of risk.

### TODO: Setup a MyEpsilonSamplingBanditModel instance
First, construct [a `MyEpsilonSamplingBanditModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyEpsilonSamplingBanditModel), which holds information about the [ϵ-greedy sampling approach](https://arxiv.org/abs/1707.02038). Given that the risk-aware modifications occur in the response of the world, i.e., in the feedback that an agent receives by implementing an action, we can reuse the bandit model saved from the example. 
* Retrieve the [`MyEpsilonSamplingBanditModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyEpsilonSamplingBanditModel) from the `saved_state_dict::Dict{String, Any}` dictionary using the `banditmodel` key. Save the bandit model in the `bandit_model::MyEpsilonSamplingBanditModel` variable:

In [19]:
bandit_model = saved_state_dict["banditmodel"];

### TODO: Setup a MyTickerPickerRiskAwareWorldModel instance
Now that we have the `bandit_model::MyEpsilonSamplingBanditModel` instance construct [a `MyTickerPickerRiskAwareWorldModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyTickerPickerRiskAwareWorldModel) which holds data about the world, and the `world(...)` function that our agents sample. 
* The [`MyTickerPickerRiskAwareWorldModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyTickerPickerRiskAwareWorldModel) holds similar data to the worked example, with the expectation that we now have an additional field which holds risk information, this is the `risk::Dict{String, Float64}` field; the `keys` are the `ticker`$\in$ `my_tickers` symbols and the `values` are risk measures (in this case the $\beta_{i}$ values).
* Like before, the `world(...)` function takes problem data and returns a binary reward. However, in this case, we refactor the `world(...)` function, which takes the `start::Int64` index, an action $a\in\mathcal{A}$ value, and the `worldmodel::MyTickerPickerRiskAwareWorldModel` to account for risk. The `world(...)` function returns a binary reward $r\in\left\{0,1\right\}$.

#### Implement the risk-award world function
`Unhide` the code block and complete the risk-aware implementation of the `world(...)` function. 
* Let's compare the wealth generated by investing `1 USD` in `SPY` for `buffersize::Int64` periods of $\Delta{t}$ duration (denoted as `W_SPY`) versus `1 USD` in a `ticker` over the same period (denoted as `W_ticker`). If `W_ticker >= W_SPY`, the `world(...)` function returns `1`, i.e., success; otherwise, the `world(...)` function returns `0.`
* __Small logic_change__: In the reward decision logic, we use equality rather than inequality. Therefore, the selection of `ticker` over `SPY` results in a `+1` reward if `ticker` is at least as good as `SPY`. This addresses a situation where `ticker` equals `SPY`; an inequality would consider this to be a loss.

In [21]:
function world(start::Int64, action::Int64, worldmodel::MyTickerPickerRiskAwareWorldModel)::Int64
    
    # initialize 
    tickers = worldmodel.tickers;
    data = worldmodel.data;
    risk_free_rate = worldmodel.risk_free_rate;
    Δt = worldmodel.Δt;
    buffersize = worldmodel.buffersize;
    risk_dictionary = worldmodel.risk;
    time = range(start+1,(start+buffersize), step = 1) |> collect

    # What is the ticker?
    ticker_symbol = tickers[action]; # ticker we are looking at?
    if (ticker_symbol == "SPY")
        return 1; # save some computation, this case will always return 1
    end

    # Compute the wealth by investing in SPY
    W_SPY = 0.0;
    SPY_price_df = data["SPY"];
    buffer_SPY = Array{Float64,1}();
    for t ∈ time
        P₁ = SPY_price_df[t-1, :volume_weighted_average_price]
        P₂ = SPY_price_df[t, :volume_weighted_average_price]
        R = (1/Δt)*log(P₂/P₁); # growth rate SPY
        push!(buffer_SPY,R);
    end
    μ_SPY = sum(buffer_SPY);
    W_SPY = exp(μ_SPY*Δt); # if we invested 1 USD in SPY, how much at the end of the period (risk-adjusted)
    
    # Compute the wealth by investing in ticker
    W_ticker = 0.0;
    price_df = data[ticker_symbol];
    buffer_ticker = Array{Float64,1}();
    for t ∈ time
        P₁ = price_df[t-1, :volume_weighted_average_price]
        P₂ = price_df[t, :volume_weighted_average_price]
        R = (1/Δt)*log(P₂/P₁); # growth rate ticker 
        push!(buffer_ticker,R);
    end
    β = risk_dictionary[ticker_symbol];
    sum_value = sum(buffer_ticker);
    γ = buffersize*sign(sum_value);
    μ̄ = (β^(-1*γ))*sum_value;
    W_ticker = exp(μ̄*Δt); # if we invested 1 USD in ticker, how much at the end of the period (risk-adjusted)
 
    # Are we better or worse relative to the risk-free investment?
    result_flag = nothing;
    (W_ticker >= W_SPY) ? result_flag = 1 : result_flag = 0   
    
    # return the reward -
    return result_flag;
end;

Next, set the `buffersize::Int64` variable. Use the same value as the worked example so that we can compare the risk-blind and risk-aware result:

In [23]:
buffersize = saved_state_dict["buffersize"]; # use the value from the worked example

Finally, construct a [`MyTickerPickerRiskAwareWorldModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyTickerPickerRiskAwareWorldModel) using 
[a custom `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.build-Tuple{Type{MyTickerPickerRiskAwareWorldModel},%20NamedTuple}). The [`build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.build-Tuple{Type{MyTickerPickerRiskAwareWorldModel},%20NamedTuple}) takes two arguments: first, the [`MyTickerPickerRiskAwareWorldModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyTickerPickerRiskAwareWorldModel) and the second is a [Julia `NamedTuple` instance](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) holding data for the model:
* The [`MyTickerPickerRiskAwareWorldModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyTickerPickerRiskAwareWorldModel) requires the `my_tickers::Array{String,1}` array, the `data::Dict{String, DataFrame}` field holding the price data dictionary, the `world::Function`, the `risk_free_rate::Float64`, the duration of the time-step $\Delta{t}$,  the `buffersize::Int64` and the `risk::Dict{String, Float64}` fields. Store the world model in the `risk_aware_world_model::MyTickerPickerWorldModel` variable:

In [25]:
risk_aware_world_model = build(MyTickerPickerRiskAwareWorldModel, (
    tickers = my_tickers,
    data = dataset,
    world = world,
    risk_free_rate = risk_free_rate,
    Δt = Δt,
    buffersize = buffersize,
    risk = β
));

## Task 3: Compute the preferences of a population risk-aware ticker-picker agents
In this task, explore the preference of $N = 10000$ risk-aware agents by running the [`sample(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.sample-Tuple{MyEpsilonSamplingBanditModel,%20MyTickerPickerWorldModel}) inside a [`for-loop`](https://docs.julialang.org/en/v1/base/base/#for). Store the results at a particular `trading_day_index::Int64` in the `risk_aware_agent_specific_data::Array{Beta,2}` array. 
* The `risk_aware_agent_specific_data::Array{Beta,2}` array holds the [`Beta` distributions](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.Beta) for each agent at `trading_day_index::Int64` for each `ticker`$\in$ `my_tickers.` The `ticker` is the row index, and the agent's [`Beta` distributions](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.Beta) is the column index, e.g., index (1,4) holds the preference for ticker = 1 held by agent = 4 at time `trading_day_index::Int64.`
* In the code block below, use [the Julia `foreach(...)` iteration pattern](https://docs.julialang.org/en/v1/base/collections/#Base.foreach) to populate the `risk_aware_agent_specific_data::Array{Beta,2}` array. You can also use [a simple `for-loop`](https://docs.julialang.org/en/v1/base/base/#for), if you wish.

In [27]:
trading_day_index = (maximum_number_trading_days - buffersize); # What trading day are we interested in?
risk_aware_agent_specific_data = Array{Beta,2}(undef, K, number_of_agents);
for agent_index ∈ 1:number_of_agents
    
    time_sample_results_dict_eps = VLQuantitativeFinancePackage.sample(bandit_model, risk_aware_world_model; 
        horizon = (maximum_number_trading_days - buffersize)); # sample

    # populate the agent_specific_data array
    foreach(k-> risk_aware_agent_specific_data[k, agent_index] = time_sample_results_dict_eps[trading_day_index][k], 1:K)
end

### TODO: Compute the wisdom of the risk-aware collective
Now that we have preferences for the population of agents, compute a consensus belief of which tickers we might include in your portfolio $\mathcal{P}$. First, compute the agent-specific rank of each `ticker`$\in$`my_tickers,` where `rank = 1` is the best, and `rank = K` is the worst. We'll store these values in the `risk_aware_preference_rank_array::Array{Int,2}` array.

In [29]:
risk_aware_preference_rank_array = Array{Int,2}(undef, number_of_agents, K);
for agent ∈ 1:number_of_agents
        
    # ask an agent about their preference for ticker i -
    experience_distributions = risk_aware_agent_specific_data[:,agent]
    preference_vector = preference(experience_distributions, my_tickers) .|> x-> trunc(Int64, x) # wow the trunc function is cool!

    # package
    foreach(i -> risk_aware_preference_rank_array[agent, i] = preference_vector[i], 1:K);
end
risk_aware_preference_rank_array

10000×29 Matrix{Int64}:
 26   8  21  21  14  14  26  18   3  14  …  26   8  14  1  21   5  26  14  26
 18  18  10  26   2  26  13  18   7  26     18  10  10  1  26  10  18  18  18
 22  22  28  11   6  11  22  15   4  28     22   3  15  1  22  11  22  22  22
 26  12  26  19   3   8  12  19  12  19     16   6  26  1  26  12  12  12  19
 21  21  21  21  10  21  21  10  21   5     21  10  21  1  21  10  21  21  21
 19  19  11  19   7  27  19  19   5  27  …  19  11  19  1  27  19  19   3  19
 20  20  20  20   5  28  20  12  20  20     20   3  28  1   8   6  20   4  12
 20  20  20  20   6  20   9  20  20  20     20   5  29  1   9  20  20  20  20
 10  19  19  19  19  28  19  19   7  10     19   3  19  1  28  19  19   5  19
 20  20  20  28   4  20  20  20  20   7     20   5  20  1  20   7  20  20  20
 29  20  20  20   9  20  20   9  20  20  …  20   6  11  1  20  20  20   5  20
 20  20  20  28  20  20  20  20   2  20     28  20  20  1  20  20  20   9  11
 19  19  19  19  11  19  19  11   8  19 

#### Compute the risk-aware probability dictionary
Count the number of times a `ticker` is ranked in the `top M` tickers across the population of agents and then normalize by the number of agents, i.e., estimate the probability of being ranked in the `top M` using a [`for-loop`](https://docs.julialang.org/en/v1/base/base/#for).
* Set the value `M::Int64`, the number of top firms we consider, from the `saved_state_dict::Dict{String, Any}.` Then, initialize data storage for the risk-aware probability values in the `risk_aware_probability_top_dictionary::Dict{String, Float64}` dictionary, where the `ticker::String` is the key, and the frequency is the value.
* For each ticker index `i`, compute the risk-aware probability, and then store this value in the `risk_aware_probability_top_dictionary::Dict{String, Float64}` dictionary.

In [31]:
M = saved_state_dict["M"]; # load same top M as worked example
risk_aware_probability_top_dictionary = Dict{String,Float64}();
for i ∈ eachindex(my_tickers)
    probability = findall(x-> x ≤ M, risk_aware_preference_rank_array[:,i]) |> x -> length(x) |> x-> x/number_of_agents; # compute probability
    my_tickers[i] |> ticker -> risk_aware_probability_top_dictionary[ticker] = probability; # store probability
end

### TODO: Preference table for risk-blind and risk-aware agents
The `risk_aware_probability_top_dictionary::Dict{String, Float64}` holds the probability that a `ticker`$\in$ `my_tickers` is ranked in the `top M.` Let's build a table of these values (and the associated ordinal rankings) and contrast them with the `SPY`-only rankings

#### Summary
`Unhide` the code block below to see how we constructed a table holding the probability that a `ticker`$\in$ `my_tickers` is ranked in the `top M` firms and the corresponding ordinal rank of each firm. This calculation uses [the `ordinalrank(...)` function exported by the StatsBase.jl package](https://github.com/JuliaStats/StatsBase.jl) and [the PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl).
* This table shows data for two cases. First, we just performed a simulation, looking at the choice between a ticker and SPY alone in the presence of risk; this will be the `risk_aware` data. However, for comparison, we also computed (in an outside simulation) the same choice, i.e., ticker versus SPY, _without_ the risk correction; this will be the `risk_blind` data.
* The difference in ranking between the risk-blind values and the risk-aware values is shown in the `Δ` column, where $\Delta$ = `risk_aware` - `risk_blind.` Therefore, if $\Delta > 0$, the ranking in the risk-aware case is higher than in the risk-blind case (the higher the rank, the less attractive the ticker). On the other hand, if $\Delta < 0$, the ranking decreases in the risk-aware case, meaning the ticker is perceived as a better choice by the population of agents (relative to SPY) when risk is considered.
* As the risk parameter $\beta\uparrow$, the risk-aware ranking generally increases $\uparrow$, i.e., agents are more sensitive to high $\beta$ tickers and prefer these firms (or ETFs) less. For example, `JNJ` has a $\beta\sim$ 0.5, i.e., `JNJ` is half as risky as `SPY.` Thus, risk-aware agents prefer `JNJ` to high-$\beta$ tickers such as `NVDA` or `TSLA` (approximately `1.7`$\times$ more risky than `SPY`) because of its decreased volatility.
* Anomalies observed in the project, such as high-beta stocks not responding to risk, for example, `OXY` or `MU,` are not seen here. In this case, the preferences for both `OXY` and `MU` decrease when using `SPY` as the alternative investment.

In [33]:
let
    # initialize 
    df = DataFrame();
    tmp_risk_blind = Array{Float64,1}();
    tmp_risk_aware = Array{Float64,1}();

    # compute the ordinal rank of the sorted list of tickers
    my_sorted_tickers = my_tickers |> sort;

    # blind -
    foreach(ticker -> push!(tmp_risk_blind, risk_blind_probability_top_dictionary[ticker]), my_sorted_tickers)
    ordinal_rank_risk_blind = ordinalrank(tmp_risk_blind, rev=true); # function exported by StatsBase.jl

    # aware -
    foreach(ticker -> push!(tmp_risk_aware, risk_aware_probability_top_dictionary[ticker]), my_sorted_tickers)
    ordinal_rank_risk_aware = ordinalrank(tmp_risk_aware, rev=true); # function exported by StatsBase.jl
    
    # populate rows in the df -
    for i ∈ eachindex(my_sorted_tickers)
        ticker_value = my_sorted_tickers[i];
        Δ = (ordinal_rank_risk_aware[i] - ordinal_rank_risk_blind[i])
        if (Δ > 0)
            pref_w_risk = "↓"
        elseif (Δ < 0)
            pref_w_risk = "↑"
        else
            pref_w_risk = "↔"
        end
        
        row_df = (
            ticker = ticker_value,
            β = β[ticker_value],
            p_risk_blind = risk_blind_probability_top_dictionary[ticker_value],
            p_risk_aware = risk_aware_probability_top_dictionary[ticker_value],
            r_risk_blind = ordinal_rank_risk_blind[i],
            r_risk_aware = ordinal_rank_risk_aware[i],
            Δ = Δ,
            Δ_pref_w_risk = pref_w_risk
        );
        push!(df, row_df);
    end
    pretty_table(df); 
end

┌────────┬──────────┬──────────────┬──────────────┬──────────────┬──────────────┬───────┬───────────────┐
│ ticker │        β │ p_risk_blind │ p_risk_aware │ r_risk_blind │ r_risk_aware │     Δ │ Δ_pref_w_risk │
│ String │  Float64 │      Float64 │      Float64 │        Int64 │        Int64 │ Int64 │        String │
├────────┼──────────┼──────────────┼──────────────┼──────────────┼──────────────┼───────┼───────────────┤
│   AAPL │  1.24019 │       0.3789 │       0.0937 │            7 │           20 │    13 │             ↓ │
│    AMD │  1.67809 │       0.4001 │       0.1186 │            6 │           19 │    13 │             ↓ │
│    BAC │  1.32778 │       0.3602 │       0.0693 │           10 │           22 │    12 │             ↓ │
│      C │  1.48297 │        0.227 │       0.0355 │           22 │           29 │     7 │             ↓ │
│  CMCSA │ 0.856908 │       0.1744 │       0.5359 │           28 │           11 │   -17 │             ↑ │
│    ECL │  1.13742 │       0.3249 │       0.2

## Task 4: Save some data for later
In the final advanced example of this module, we will build a portfolio by sampling the ticker-picker agents. Let's store some data from this simulation tforthe final advanced example. We'll use the [save(...) method exported by the JLD2.jl package](https://github.com/JuliaIO/JLD2.jl.git) to write a [file in HDF5 binary format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format). First, we specify a `path` in the `path_to_save_file` variable: 

In [35]:
path_to_save_file = joinpath(_PATH_TO_DATA, "TickerPickerAgent-Simulation-State-ALT-RA-SPY-N-$(number_of_agents).jld2");

Then we write an [`HDF5 binary file`](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) holding our data to the location specified by `path_to_save_file.` We use [the `save(...)` function exported by the JLD2.jl package to write a binary save file](https://github.com/JuliaIO/JLD2.jl.git) (later we'll use to the `load(...)` function to reload this data):

In [37]:
save(path_to_save_file, Dict("preferences_risk_aware" => risk_aware_agent_specific_data, 
        "preferences_risk_blind" => risk_blind_preferences,
        "insampledata" => dataset,
        "tickers" => risk_aware_world_model.tickers));

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.